In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

import backtrader as bt
import datetime
import os.path
import sys
%matplotlib inline

In [2]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        #adding sma indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)       
    
    def notify_order(self, order):
        # Buy/Sell order submitted/accepted to/by broker - Nothing to do
        if order.status in [order.Submitted, order.Accepted]:
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (order.executed.price, 
                                                                               order.executed.value, 
                                                                               order.executed.comm))
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (order.executed.price, 
                                                                                order.executed.value, 
                                                                                order.executed.comm))
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
    
    
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
    
    
    
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
                # current price greater than the sma

                # BUY, BUY, BUY!!! (with default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=20)

        else:

            # Already in the market ... we might sell
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

In [3]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    #add a test strategy
#     cerebro.addstrategy(TestStrategy)
    strats = cerebro.optstrategy(
        TestStrategy, 
        maperiod=range(10,31)
    )
    
    startdate = datetime.datetime(2020, 1, 1)
    enddate = datetime.datetime(2020, 7, 31)
    
    # Create a Data Feed
    data = bt.feeds.YahooFinanceData(dataname = 'SBIN.NS',
                                        fromdate = startdate,
                                        todate = enddate, reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    
    #setting value of initial cash
    cerebro.broker.setcash(100000.0)
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    #stdstats parameter removes stats data while plotting
    cerebro.run(stdstats=False, maxcpus=1)

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # Plot the result
#     cerebro.plot()

Starting Portfolio Value: 100000.00
2020-07-30, (MA Period 10) Ending Value 98763.47
2020-07-30, (MA Period 11) Ending Value 98926.67
2020-07-30, (MA Period 12) Ending Value 99066.50
2020-07-30, (MA Period 13) Ending Value 99119.93
2020-07-30, (MA Period 14) Ending Value 99148.13
2020-07-30, (MA Period 15) Ending Value 99092.19
2020-07-30, (MA Period 16) Ending Value 99134.25
2020-07-30, (MA Period 17) Ending Value 99164.78
2020-07-30, (MA Period 18) Ending Value 99307.99
2020-07-30, (MA Period 19) Ending Value 99307.99
2020-07-30, (MA Period 20) Ending Value 99307.99
2020-07-30, (MA Period 21) Ending Value 99568.31
2020-07-30, (MA Period 22) Ending Value 99650.72
2020-07-30, (MA Period 23) Ending Value 99390.67
2020-07-30, (MA Period 24) Ending Value 99390.67
2020-07-30, (MA Period 25) Ending Value 99447.45
2020-07-30, (MA Period 26) Ending Value 99506.51
2020-07-30, (MA Period 27) Ending Value 99506.51
2020-07-30, (MA Period 28) Ending Value 99572.05
2020-07-30, (MA Period 29) Ending